In [1]:
#import libraries
import pandas as pd
import tweepy
from tweepy import OAuthHandler
import string

import spacy #load spacy
nlp = spacy.load("en", disable=['parser', 'tagger', 'ner'])
stops = nlp.Defaults.stop_words


In [3]:
#twitter credentials
consumer_key = 'a4KB4gNSf4ws5auJNt0rcszsR'
consumer_secret = 'a0Gd2o0xACrUm2Qj6iwiNht2JA1WnoXeAzDXH6uuxx9qvCa0hw'
access_token = '218794533-uwE3tbCBFKd08Hnp9cZIxuoGyFHvn7d6QSAkap2L'
access_secret = 'ecqDFN020aDhotvZ5LP0wYMjllx7lNEo4OabJrbNpSIAH'

#Authenticate credentials
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth)

tweet_created = []
tweet_text = []
tweets = tweepy.Cursor(api.search,q='#covid19',count=10, lang="en",until='2020-04-23')
for tweet in tweets.items():
    tweet_created.append(tweet.created_at)
    tweet_text.append(tweet.text)  

df = pd.DataFrame(columns=['created_on','tweets'])
df['created_on'] = tweet_created
df['tweets']= tweet_text

TweepError: Twitter error response: status code = 429

## Cleaning Data:
1. Removing hastags, cc, retweets, mentions, emojis etc. 
2. removing stop words
3. convert into lower case
4. Lemmatization


In [ ]:
#convert tweets to lower case
df['preprocess_data'] = df['tweets'].str.lower()
#url removes
df['preprocess_data'] = df['preprocess_data'].str.replace(r'(https|http)?:\/(\w|\.|\/|\?|\=|\&|\%)*\b','')
df['preprocess_data'] = df['preprocess_data'].str.replace(r'www\.\S+\.com','')


#removes retweets & cc
df['preprocess_data'] = df['preprocess_data'].str.replace(r'rt|cc', '')

#hashtags removes
df['preprocess_data'] = df['preprocess_data'].str.replace(r'#\S+', '')

#user mention removes
df['preprocess_data'] = df['preprocess_data'].str.replace(r'@\S+', '')

#emoji 
df['preprocess_data'] = df['preprocess_data'].str.replace(r'[^\x00-\x7F]+', '')

#html tags
df['preprocess_data'] = df['preprocess_data'].str.replace(r'<.*?>', '')

#removes extra spaces
df['preprocess_data'] = df['preprocess_data'].str.replace(r' +', ' ')

#punctuation
df['preprocess_data'] = df['preprocess_data'].str.replace('[{}]'.format(string.punctuation), '')



In [ ]:
#  Funtion to convert into lowercase, remove stop words
    def normalize(comment, lowercase, remove_stopwords):
    if lowercase:
        comment = comment.lower()
    comment = nlp(comment)
    lemmatized = list()
    for word in comment:
        lemma = word.lemma_.strip()
        if lemma:
            if not remove_stopwords or (remove_stopwords and lemma not in stops):
                lemmatized.append(lemma)
    return " ".join(lemmatized)

In [ ]:
# Apply function on the data frame text col
df['tweets_clean'] = df['preprocess_data'].apply(normalize, lowercase=True, remove_stopwords=True)

In [ ]:
df.head()

In [ ]:
#convert preprocessed list words to string 
df['preprocess_str'] = df['tweets_clean'].apply(' '.join)
df.head()

# Sentiments for the tweets

In [ ]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [ ]:
sid = SentimentIntensityAnalyzer()

In [ ]:
# Getting polarity scores of full dataset
df['scores'] = df['preprocess_data'].apply(lambda x: sid.polarity_scores(x))

In [ ]:
# Getting only the final compound score as the overall score
df['compound'] = df['scores'].apply(lambda d:d['compound'])

In [ ]:
# Create postive or negative cat of the compound score
df['comp_score'] = df['compound'].apply(lambda score: 'pos' if score >=0 else 'neg')

In [ ]:
df.head()

In [ ]:
df.comp_score.value_counts()